In [1]:
import pandas as pd

from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/strokes/stroke-real.csv')

In [3]:
df.drop(columns='id', inplace=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df_train, df_test=train_test_split(df,test_size=0.4, random_state=42)
df_train.shape, df_test.shape

((3066, 11), (2044, 11))

In [5]:
predictor= TabularPredictor(
    label='stroke',
    problem_type='binary',
    eval_metric='accuracy',
    path='AutogluonModels/strokes/real/v2',
    learner_kwargs = {'positive_class':1}
    )

In [6]:
predictor.fit(
    train_data = df_train,
    verbosity = 2,
    presets=['best_quality'],
    time_limit = 900
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       9.85 GB / 15.94 GB (61.8%)
Disk Space Avail:   121.14 GB / 446.36 GB (27.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up 

In [7]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                          model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           WeightedEnsemble_L2   0.957599    accuracy       0.047842   2.815633                0.000000           0.333111            2       True         47
1          LightGBM_r130_BAG_L1   0.957599    accuracy       0.047842   2.482522                0.047842           2.482522            1       True         31
2          CatBoost_r167_BAG_L1   0.957273    accuracy       0.022014  13.454717                0.022014          13.454717            1       True         46
3           CatBoost_r50_BAG_L1   0.957273    accuracy       0.025862  14.646904                0.025862          14.646904            1       True         33
4           CatBoost_r70_BAG_L1   0.957273    accuracy       0.027936  25.316712                0.027936          25.316712            1       

c:\Users\jgala\uned\tfm\synthetic-data\.venv\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetTorch_r79_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiT

In [8]:
predictor.evaluate(df_test)

c:\Users\jgala\uned\tfm\synthetic-data\.venv\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.9442270058708415,
 'balanced_accuracy': 0.5,
 'mcc': 0.0,
 'roc_auc': 0.8325606763021544,
 'f1': 0.0,
 'precision': 0.0,
 'recall': 0.0}

In [9]:
predictor.leaderboard(df_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,1.000000,0.951729,accuracy,0.030048,0.019974,0.011417,0.030048,0.019974,0.011417,1,True,2
1,RandomForest_r195_BAG_L1,1.000000,0.952381,accuracy,0.115819,0.119380,0.939562,0.115819,0.119380,0.939562,1,True,26
2,ExtraTrees_r42_BAG_L1,1.000000,0.953686,accuracy,0.125780,0.127156,0.864105,0.125780,0.127156,0.864105,1,True,22
3,RandomForestGini_BAG_L1,1.000000,0.954664,accuracy,0.142690,0.134330,0.885876,0.142690,0.134330,0.885876,1,True,5
4,RandomForestEntr_BAG_L1,1.000000,0.955316,accuracy,0.143310,0.110959,0.625150,0.143310,0.110959,0.625150,1,True,6
5,ExtraTreesGini_BAG_L1,1.000000,0.954012,accuracy,0.190141,0.126427,0.644592,0.190141,0.126427,0.644592,1,True,8
6,ExtraTreesEntr_BAG_L1,1.000000,0.954338,accuracy,0.190220,0.127923,0.613879,0.190220,0.127923,0.613879,1,True,9
7,RandomForest_r39_BAG_L1,0.979778,0.954338,accuracy,0.126749,0.118204,0.895854,0.126749,0.118204,0.895854,1,True,45
8,CatBoost_r9_BAG_L1,0.962492,0.956621,accuracy,0.101517,0.030594,31.320216,0.101517,0.030594,31.320216,1,True,18
9,CatBoost_r70_BAG_L1,0.961840,0.957273,accuracy,0.113257,0.027936,25.316712,0.113257,0.027936,25.316712,1,True,42


In [10]:
predictor.feature_importance(data=df_train)

Computing feature importance via permutation shuffling for 10 features using 3066 rows with 5 shuffle sets...
	4.99s	= Expected runtime (1.0s per shuffle set)
	2.34s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
gender,0.0,0.0,0.5,5,0.0,0.0
age,0.0,0.0,0.5,5,0.0,0.0
hypertension,0.0,0.0,0.5,5,0.0,0.0
heart_disease,0.0,0.0,0.5,5,0.0,0.0
ever_married,0.0,0.0,0.5,5,0.0,0.0
work_type,0.0,0.0,0.5,5,0.0,0.0
Residence_type,0.0,0.0,0.5,5,0.0,0.0
avg_glucose_level,0.0,0.0,0.5,5,0.0,0.0
bmi,0.0,0.0,0.5,5,0.0,0.0
smoking_status,0.0,0.0,0.5,5,0.0,0.0
